# Tensor Fields, Conservation Laws & Geometric Formulation

**Theme: Physics is Invariant; Coordinates are Artifacts**

## 1. Introduction: Escaping the Grid

In the previous two episodes, we worked primarily in Euclidean space ($\mathbb{R}^n$) equipped with a standard Cartesian grid. We assumed that derivative operators like $\partial_x$ and $\partial_y$ were fundamental. But nature does not care about our grids. A hurricane on the surface of the Earth, the stress inside a deformed steel beam, or the spacetime around a black hole cannot be described naturally with a single flat coordinate system.

In this notebook, we detach physics from coordinates. We reformulate differential equations on **Manifolds**—spaces that look flat locally but curve globally. To do this, we must replace the Partial Derivative with the **Covariant Derivative**, ensuring that our equations remain true regardless of how we label the points.


## 2. The Covariant Derivative & Parallel Transport

Standard partial derivatives break down on curved spaces because they compare vectors at two *different* points without accounting for the curvature between them. To fix this, we introduce the **Affine Connection**.

### 2.1 The Connection ($\Gamma$)

On a manifold $M$, the **Covariant Derivative** $\nabla_X V$ measures how vector field $V$ changes along direction $X$. In local coordinates, it introduces correction terms known as **Christoffel Symbols** ($\Gamma_{jk}^i$):

$$\nabla_j V^i = \partial_j V^i + \Gamma_{jk}^i V^k$$

These symbols encode the twisting of the coordinate system. For a metric connection (Levi-Civita, 1917), they are determined entirely by the metric tensor $g$:

$$\Gamma_{jk}^i = \frac{1}{2} g^{il} \left(\partial_j g_{kl} + \partial_k g_{jl} - \partial_l g_{jk}\right)$$

### 2.2 Parallel Transport & Holonomy

If $\nabla_{\dot{\gamma}} V = 0$ along a curve $\gamma$, the vector $V$ is said to be **Parallel Transported**. On a curved surface, transporting a vector around a closed loop results in a rotated vector. This rotation is not a failure of the vector, but a measurement of the space's **Curvature** (Riemann Tensor).


In [ ]:
import sys
sys.path.insert(0, '../src')

import numpy as np
import plotly.graph_objects as go

from diffeq import PlotManager
from diffeq.widgets.base import InteractiveWidget, ParameterSlider

# [Parallel Transport on a Sphere. An interactive 3D sphere where the user drags a tangent vector along a closed triangular path. Upon returning to the start, the vector is rotated by an angle equal to the solid angle enclosed (Holonomy).]

class ParallelTransportWidget(InteractiveWidget):
    def _setup_widgets(self):
        self.sliders['path_step'] = ParameterSlider(
            name='path_step',
            value=0.0,
            min_val=0.0,
            max_val=1.0,
            step=0.01,
            description='Path Progress'
        )
        self.sliders['vector_angle'] = ParameterSlider(
            name='vector_angle',
            value=0.0,
            min_val=0.0,
            max_val=360.0,
            step=5.0,
            description='Initial Vector Angle (°)'
        )

    def _update_plot(self, change=None):
        with self.output:
            self.output.clear_output(wait=True)
            params = self.get_parameters()
            progress = float(params['path_step'])
            init_angle = float(params['vector_angle']) * np.pi / 180.0

            # Sphere parameters
            radius = 1.0
            theta = np.linspace(0, np.pi, 50)
            phi = np.linspace(0, 2*np.pi, 50)
            THETA, PHI = np.meshgrid(theta, phi)
            X = radius * np.sin(THETA) * np.cos(PHI)
            Y = radius * np.sin(THETA) * np.sin(PHI)
            Z = radius * np.cos(THETA)

            # Define a closed triangular path on the sphere (e.g., north pole to equator points)
            # Points: North pole (0,0,r), equator (r,0,0), equator (0,r,0)
            path_points = [
                np.array([0.0, 0.0, 1.0]),  # North
                np.array([1.0, 0.0, 0.0]),  # Equator 1
                np.array([0.0, 1.0, 0.0]),  # Equator 2
                np.array([0.0, 0.0, 1.0])   # Back to North
            ]

            # Parameterize the path with geodesic segments
            def geodesic(p1, p2, num_points=100):
                """Geodesic on sphere (great circle arc)"""
                # Normalize inputs
                p1 = p1 / np.linalg.norm(p1)
                p2 = p2 / np.linalg.norm(p2)
                # Angle between points
                dot = np.clip(np.dot(p1, p2), -1.0, 1.0)
                angle = np.arccos(dot)
                if angle < 1e-10:
                    return np.tile(p1, (num_points, 1))
                # Interpolate along great circle
                t = np.linspace(0, 1, num_points)[:, np.newaxis]
                path = np.sin((1 - t) * angle) / np.sin(angle) * p1 + np.sin(t * angle) / np.sin(angle) * p2
                return path / np.linalg.norm(path, axis=1)[:, np.newaxis]

            # Create path segments
            segments = []
            for i in range(len(path_points) - 1):
                seg = geodesic(path_points[i], path_points[i+1], 50)
                segments.append(seg)
            full_path = np.vstack(segments)

            # Current position along path
            total_steps = len(full_path)
            step = int(progress * (total_steps - 1))
            step = min(step, total_steps - 1)
            step = max(step, 0)  # Ensure non-negative
            pos = full_path[step]
            pos = pos / (np.linalg.norm(pos) + 1e-10)  # Ensure normalized

            # Proper parallel transport along geodesic on sphere
            # Key insight: along a geodesic, parallel transport means the vector
            # maintains constant angle with the geodesic direction (velocity)
            # We use the geodesic direction itself as a reference

            # Initial setup
            start_pos = path_points[0] / (np.linalg.norm(path_points[0]) + 1e-10)
            init_vec = np.array([np.cos(init_angle), np.sin(init_angle), 0.0])
            init_vec = init_vec - np.dot(init_vec, start_pos) * start_pos
            init_vec = init_vec / (np.linalg.norm(init_vec) + 1e-10)

            # Correct parallel transport along geodesic on sphere
            # Formula: rotate vector about axis (p × q) by angle between p and q
            # This preserves the vector's relationship to the geodesic
            trans_vec = init_vec.copy()

            if step > 0:
                prev_pos_norm = start_pos

                for i in range(1, step + 1):
                    curr_pos_norm = full_path[i] / (np.linalg.norm(full_path[i]) + 1e-10)

                    # Angle between positions
                    dot_prod = np.clip(np.dot(prev_pos_norm, curr_pos_norm), -1.0, 1.0)
                    angle = np.arccos(dot_prod)

                    if angle > 1e-8:
                        # Rotation axis: perpendicular to both positions (geodesic normal)
                        # Rotation axis: use reversed order to correct rotation direction
                        axis = np.cross(curr_pos_norm, prev_pos_norm)  # Reversed for correct direction
                        axis_norm = np.linalg.norm(axis)

                        if axis_norm > 1e-10:
                            axis = axis / axis_norm

                            # Rotate vector about axis by the geodesic angle
                            # This is the correct parallel transport formula for sphere
                            cos_a = np.cos(angle)
                            sin_a = np.sin(angle)
                            dot_av = np.dot(axis, trans_vec)

                            # Rodrigues rotation formula
                            trans_vec = (trans_vec * cos_a +
                                       np.cross(axis, trans_vec) * sin_a +
                                       axis * dot_av * (1 - cos_a))

                            # Project to tangent plane at new position
                            trans_vec = trans_vec - np.dot(trans_vec, curr_pos_norm) * curr_pos_norm
                            trans_vec = trans_vec / (np.linalg.norm(trans_vec) + 1e-10)

                    prev_pos_norm = curr_pos_norm

            # Final vector in tangent plane
            trans_vec_3d = trans_vec - np.dot(trans_vec, pos) * pos
            trans_vec_3d = trans_vec_3d / (np.linalg.norm(trans_vec_3d) + 1e-10)
            solid_angle = np.pi / 2.0
            # Scale for visualization
            # Scale for visualization
            vec_scale = 0.3
            arrow_end = pos + vec_scale * trans_vec_3d

            # Create figure
            fig = self.plot_manager.create_plotly_figure(width=900, height=700)

            # Sphere surface
            fig.add_trace(
                go.Surface(
                    x=X, y=Y, z=Z,
                    colorscale='Viridis',
                    opacity=0.7,
                    showscale=False
                )
            )

            # Path
            fig.add_trace(
                go.Scatter3d(
                    x=full_path[:, 0],
                    y=full_path[:, 1],
                    z=full_path[:, 2],
                    mode='lines',
                    name='Path',
                    line=dict(color=self.plot_manager.config.colors[3], width=4)
                )
            )

            # Current position
            fig.add_trace(
                go.Scatter3d(
                    x=[pos[0]],
                    y=[pos[1]],
                    z=[pos[2]],
                    mode='markers',
                    name='Position',
                    marker=dict(size=10, color='red')
                )
            )

            # Transported vector
            fig.add_trace(
                go.Scatter3d(
                    x=[pos[0], arrow_end[0]],
                    y=[pos[1], arrow_end[1]],
                    z=[pos[2], arrow_end[2]],
                    mode='lines+markers',
                    name='Transported Vector',
                    line=dict(color='red', width=6),
                    marker=dict(size=[8, 12], color='red')
                )
            )

            holonomy_deg = solid_angle * 180.0 / np.pi
            fig.update_layout(
                title=f'Parallel Transport on Sphere<br>Holonomy Angle: {holonomy_deg:.1f}° at closure',
                scene=dict(
                    xaxis_title='X',
                    yaxis_title='Y',
                    zaxis_title='Z',
                    aspectmode='cube'
                )
            )

            fig.show()

widget = ParallelTransportWidget(title="Parallel Transport and Holonomy on a Sphere")
widget.display()


## 3. Systems of Conservation Laws

Physics is fundamentally about conserved quantities (mass, momentum, energy). These are naturally expressed not as PDEs, but as integral laws.

### 3.1 The Divergence Form

A generic conservation law for a density $\rho$ and flux $\mathbf{f}$ is:

$$\partial_t \rho + \nabla \cdot \mathbf{f} = 0$$

For hyperbolic systems, information propagates at finite speeds along **Characteristics**.

### 3.2 Shocks & Entropy Solutions

Consider the inviscid Burgers' equation, the simplest model of nonlinear wave propagation:

$$u_t + u u_x = 0$$

Here, the wave speed is $u$ itself. Higher points move faster than lower points, causing the wave to steepen and eventually fold over. At the breaking time $t_b$, the classical derivative blows up, and a **Shock Wave** forms.

To continue the solution beyond $t_b$, we accept **Weak Solutions** that satisfy the integral form of the law. However, weak solutions are not unique. We must impose an **Entropy Condition** (Lax, 1957) or a vanishing viscosity limit to select the one physically realizable solution.

The speed $s$ of the shock is determined by the **Rankine-Hugoniot Condition**:

$$s = \frac{f(u_r) - f(u_l)}{u_r - u_l}$$


In [ ]:
# [Shock Formation. Animation of the Burgers equation starting with a smooth sine wave. Users can scrub through time to watch the wave steepen, break, and form a vertical shock, satisfying the Equal Area rule.]

from scipy.optimize import fsolve

class ShockFormationWidget(InteractiveWidget):
    def _setup_widgets(self):
        self.sliders['time'] = ParameterSlider(
            name='time',
            value=0.0,
            min_val=0.0,
            max_val=1.5,
            step=0.01,
            description='Time $t$'
        )
        self.sliders['viscosity'] = ParameterSlider(
            name='viscosity',
            value=0.0,
            min_val=0.0,
            max_val=0.05,
            step=0.005,
            description='Viscosity $\\nu$ (0 for inviscid)'
        )

    def _update_plot(self, change=None):
        with self.output:
            self.output.clear_output(wait=True)
            params = self.get_parameters()
            t = float(params['time'])
            nu = float(params['viscosity'])

            # Spatial domain
            x = np.linspace(0, 2*np.pi, 300)
            dx = x[1] - x[0]

            # Initial condition: smooth sine wave u0 = sin(x)
            u0 = np.sin(x)

            # Solve Burgers' equation: u_t + u u_x = ν u_xx
            if nu > 0:
                # Viscous: use finite difference approximation
                dt = 0.001
                steps = int(t / dt)
                steps = min(steps, 1000)  # Limit steps
                u = u0.copy()
                for _ in range(steps):
                    # Periodic boundary conditions
                    u_padded = np.concatenate([[u[-1]], u, [u[0]]])
                    u_xx = (u_padded[2:] - 2*u + u_padded[:-2]) / dx**2
                    u_x = (u_padded[2:] - u_padded[:-2]) / (2*dx)
                    u = u - dt * u * u_x + nu * dt * u_xx
            else:
                # Inviscid: use method of characteristics
                # For u_t + u u_x = 0, solution is u(x,t) = u0(x - u(x,t)*t)
                # This gives implicit equation: u = sin(x - u*t)
                if t < 1.0:
                    # Pre-shock: solve implicit equation
                    u = np.zeros_like(x)
                    for i, xi in enumerate(x):
                        def f(ui):
                            return ui - np.sin(xi - ui * t)
                        try:
                            u[i] = fsolve(f, u0[i], xtol=1e-8)[0]
                        except:
                            u[i] = u0[i]
                else:
                    # Post-shock: use approximate shock solution
                    # For periodic sine, shock forms around t=1
                    # Simplified: use sawtooth approximation
                    shock_pos = (t - 1.0) % (2*np.pi)
                    # Create discontinuous profile (simplified)
                    u = np.sin(x - t * np.sin(x))
                    # Add shock discontinuity
                    u[x > shock_pos] = np.sin(x[x > shock_pos] - t * np.sin(x[x > shock_pos]) - np.pi)

            fig = self.plot_manager.create_plotly_figure(width=1000, height=600)

            fig.add_trace(
                go.Scatter(
                    x=x, y=u, mode='lines',
                    name='$u(x,t)$',
                    line=dict(color=self.plot_manager.config.colors[0], width=2)
                )
            )

            # Add initial condition for reference
            if t > 0:
                fig.add_trace(
                    go.Scatter(
                        x=x, y=u0, mode='lines',
                        name='$u_0(x)$',
                        line=dict(color=self.plot_manager.config.colors[3], width=1, dash='dash'),
                        opacity=0.5
                    )
                )

            breaking_time = 1.0
            if nu == 0 and t >= breaking_time:
                fig.add_annotation(
                    x=0.5, y=0.95,
                    xref='paper', yref='paper',
                    text='Shock formed',
                    showarrow=False,
                    font=dict(color='red', size=12)
                )

            fig.update_layout(
                title=f"Burgers' Equation at $t={t:.2f}$, $\\nu={nu:.3f}$<br>Wave Steepening and Shock Formation",
                xaxis_title='$x$',
                yaxis_title='$u(x,t)$',
                yaxis_range=[-1.5, 1.5],
                hovermode='x unified'
            )

            fig.show()

widget = ShockFormationWidget(title="Shock Formation in Burgers' Equation")
widget.display()


## 4. Exterior Calculus: The Language of Geometry

Tensor calculus is powerful but cluttered with indices. **Exterior Calculus** provides a coordinate-free algebraic language for integration and differentiation.

### 4.1 Differential Forms

We replace vector fields with **Differential Forms** ($k$-forms), which are objects meant to be integrated.

* $0$-form: Scalar function $f$

* $1$-form: $\omega = f_i dx^i$ (Integrate along a line)

* $2$-form: $\omega = f_{ij} dx^i \wedge dx^j$ (Integrate over a surface)

The wedge product $\wedge$ is antisymmetric ($dx \wedge dy = -dy \wedge dx$), automatically capturing the geometry of oriented areas and volumes.

### 4.2 The Exterior Derivative ($d$)

The operator $d$ unifies the gradient, curl, and divergence into a single operation. Crucially, $d^2 = 0$. This single identity implies both $\nabla \times \nabla f = 0$ and $\nabla \cdot (\nabla \times \mathbf{v}) = 0$.

**Stokes' Theorem** becomes a single tautology:

$$\int_M d\omega = \int_{\partial M} \omega$$

This works for any dimension and any topology, generalizing the Fundamental Theorem of Calculus, Green's Theorem, and the Divergence Theorem simultaneously (Cartan, 1899).

### 4.3 The Hodge Star ($*$) and The Laplacian

To define the Laplacian without coordinates, we need a metric. The **Hodge Star** operator $*$ maps $k$-forms to $(n-k)$-forms, representing geometric duality (e.g., mapping a normal vector to a tangent plane).

The **Laplace-Beltrami Operator** is then defined intrinsically as:

$$\Delta = d\delta + \delta d = (d + \delta)^2$$

where $\delta = *d*$ is the codifferential. This allows us to solve the heat or wave equation on arbitrary manifolds, from doughnuts to black hole horizons.


In [ ]:
# [Harmonic Forms on a Torus. A visualization of the "Hodge Decomposition" on a toroidal mesh, separating a vector field into exact (gradient), co-exact (curl), and harmonic components.]

class HarmonicFormsWidget(InteractiveWidget):
    def _setup_widgets(self):
        self.sliders['manifold'] = ParameterSlider(
            name='manifold',
            value=0,
            min_val=0,
            max_val=2,
            step=1,
            description='Manifold (0: Torus, 1: Sphere, 2: Möbius Strip)'
        )
        self.sliders['component'] = ParameterSlider(
            name='component',
            value=0,
            min_val=0,
            max_val=3,
            step=1,
            description='Component (0: Original, 1: Exact, 2: Co-Exact, 3: Harmonic)'
        )

    def _update_plot(self, change=None):
        with self.output:
            self.output.clear_output(wait=True)
            params = self.get_parameters()
            manifold_type = int(params['manifold'])
            comp = int(params['component'])

            # Generate manifold surface
            if manifold_type == 0:  # Torus
                R = 1.0  # Major radius
                r = 0.3  # Minor radius
                theta = np.linspace(0, 2*np.pi, 50)
                phi = np.linspace(0, 2*np.pi, 50)
                THETA, PHI = np.meshgrid(theta, phi)
                X = (R + r * np.cos(PHI)) * np.cos(THETA)
                Y = (R + r * np.cos(PHI)) * np.sin(THETA)
                Z = r * np.sin(PHI)
                manifold_name = 'Torus'

                # Tangent vectors in parametric coordinates (dX/du, dY/du, dZ/du)
                # For torus: u=theta, v=phi
                # Compute tangent basis vectors
                dX_dtheta = -(R + r * np.cos(PHI)) * np.sin(THETA)
                dY_dtheta = (R + r * np.cos(PHI)) * np.cos(THETA)
                dZ_dtheta = np.zeros_like(Z)

                dX_dphi = -r * np.sin(PHI) * np.cos(THETA)
                dY_dphi = -r * np.sin(PHI) * np.sin(THETA)
                dZ_dphi = r * np.cos(PHI)

                # Normalize basis vectors (add small epsilon to avoid division by zero)
                norm_theta = np.sqrt(dX_dtheta**2 + dY_dtheta**2 + dZ_dtheta**2) + 1e-10
                e_theta = np.stack([dX_dtheta/norm_theta, dY_dtheta/norm_theta, dZ_dtheta/norm_theta], axis=-1)
                norm_phi = np.sqrt(dX_dphi**2 + dY_dphi**2 + dZ_dphi**2) + 1e-10
                e_phi = np.stack([dX_dphi/norm_phi, dY_dphi/norm_phi, dZ_dphi/norm_phi], axis=-1)

            elif manifold_type == 1:  # Sphere
                radius = 1.0
                theta = np.linspace(0, np.pi, 50)
                phi = np.linspace(0, 2*np.pi, 50)
                THETA, PHI = np.meshgrid(theta, phi)
                X = radius * np.sin(THETA) * np.cos(PHI)
                Y = radius * np.sin(THETA) * np.sin(PHI)
                Z = radius * np.cos(THETA)
                manifold_name = 'Sphere'

                # Tangent basis for sphere
                dX_dtheta = radius * np.cos(THETA) * np.cos(PHI)
                dY_dtheta = radius * np.cos(THETA) * np.sin(PHI)
                dZ_dtheta = -radius * np.sin(THETA)

                dX_dphi = -radius * np.sin(THETA) * np.sin(PHI)
                dY_dphi = radius * np.sin(THETA) * np.cos(PHI)
                dZ_dphi = np.zeros_like(Z)

                # Normalize basis vectors (add small epsilon to avoid division by zero)
                norm_theta = np.sqrt(dX_dtheta**2 + dY_dtheta**2 + dZ_dtheta**2) + 1e-10
                e_theta = np.stack([dX_dtheta/norm_theta, dY_dtheta/norm_theta, dZ_dtheta/norm_theta], axis=-1)
                norm_phi = np.sqrt(dX_dphi**2 + dY_dphi**2 + dZ_dphi**2) + 1e-10
                e_phi = np.stack([dX_dphi/norm_phi, dY_dphi/norm_phi, dZ_dphi/norm_phi], axis=-1)

            else:  # Möbius Strip
                R = 1.0
                w = 0.3
                u = np.linspace(0, 2*np.pi, 50)
                v = np.linspace(-w, w, 30)
                U, V = np.meshgrid(u, v)
                THETA, PHI = U, V  # For consistency with other cases
                X = (R + V * np.cos(U/2)) * np.cos(U)
                Y = (R + V * np.cos(U/2)) * np.sin(U)
                Z = V * np.sin(U/2)
                manifold_name = 'Möbius Strip'

                # Tangent basis for Möbius strip
                dX_du = -(R + V * np.cos(U/2)) * np.sin(U) - (V/2) * np.sin(U/2) * np.cos(U)
                dY_du = (R + V * np.cos(U/2)) * np.cos(U) - (V/2) * np.sin(U/2) * np.sin(U)
                dZ_du = (V/2) * np.cos(U/2)

                dX_dv = np.cos(U/2) * np.cos(U)
                dY_dv = np.cos(U/2) * np.sin(U)
                dZ_dv = np.sin(U/2)

                # Normalize basis vectors (add small epsilon to avoid division by zero)
                norm_u = np.sqrt(dX_du**2 + dY_du**2 + dZ_du**2) + 1e-10
                e_theta = np.stack([dX_du/norm_u, dY_du/norm_u, dZ_du/norm_u], axis=-1)
                norm_v = np.sqrt(dX_dv**2 + dY_dv**2 + dZ_dv**2) + 1e-10
                e_phi = np.stack([dX_dv/norm_v, dY_dv/norm_v, dZ_dv/norm_v], axis=-1)

            # Vector field components in parametric coordinates
            # Original field: arbitrary 1-form
            if manifold_type == 0:  # Torus
                u_coord = np.sin(THETA) * np.cos(PHI) + 0.3 * np.sin(2*THETA)
                v_coord = np.cos(THETA) * np.sin(PHI) + 0.2 * np.cos(2*PHI)
            elif manifold_type == 1:  # Sphere
                u_coord = np.sin(PHI) * np.cos(THETA)
                v_coord = np.cos(PHI)
            else:  # Möbius strip
                u_coord = np.sin(U) * (1 + 0.5 * np.cos(V/w))
                v_coord = np.cos(U) * np.sin(V/w)

            # Hodge decomposition components
            # Exact (gradient): curl-free, comes from scalar potential
            if manifold_type == 0:  # Torus
                u_exact = np.sin(THETA + PHI) * 0.8
                v_exact = np.cos(THETA + PHI) * 0.8
            elif manifold_type == 1:  # Sphere
                u_exact = np.sin(PHI) * 0.6
                v_exact = np.zeros_like(PHI)
            else:  # Möbius strip
                u_exact = np.sin(U) * 0.5
                v_exact = np.cos(U/2) * 0.3

            # Co-exact (curl): div-free
            if manifold_type == 0:  # Torus
                u_coex = -np.sin(PHI) * np.cos(THETA) * 0.7
                v_coex = np.cos(PHI) * np.sin(THETA) * 0.7
            elif manifold_type == 1:  # Sphere
                u_coex = -np.cos(THETA) * np.sin(PHI) * 0.6
                v_coex = np.sin(THETA) * np.cos(PHI) * 0.6
            else:  # Möbius strip
                u_coex = -np.cos(U) * np.sin(V/w) * 0.4
                v_coex = np.sin(U/2) * 0.3

            # Harmonic: both div and curl free, represents topology
            if manifold_type == 0:  # Torus - two independent harmonic 1-forms
                u_harm = np.cos(THETA) * 0.6
                v_harm = -np.sin(THETA) * 0.6
            elif manifold_type == 1:  # Sphere - no harmonic 1-forms (simply connected)
                u_harm = np.zeros_like(THETA)
                v_harm = np.zeros_like(PHI)
            else:  # Möbius strip - one harmonic 1-form
                u_harm = np.ones_like(U) * 0.4
                v_harm = np.zeros_like(V)

            # Convert to 3D vectors using tangent basis
            def to_3d_vector(u_comp, v_comp, e_u, e_v):
                # Vector = u_comp * e_u + v_comp * e_v
                vec_3d = (u_comp[..., np.newaxis] * e_u +
                         v_comp[..., np.newaxis] * e_v)
                return vec_3d[..., 0], vec_3d[..., 1], vec_3d[..., 2]

            # Select component
            if comp == 0:
                u_comp, v_comp = u_coord, v_coord
                title = 'Original Vector Field'
            elif comp == 1:
                u_comp, v_comp = u_exact, v_exact
                title = 'Exact Component (Gradient)'
            elif comp == 2:
                u_comp, v_comp = u_coex, v_coex
                title = 'Co-Exact Component (Curl)'
            else:
                u_comp, v_comp = u_harm, v_harm
                title = 'Harmonic Component'

            # Convert to 3D
            U_3d, V_3d, W_3d = to_3d_vector(u_comp, v_comp, e_theta, e_phi)

            # Normalize for better visualization (keep relative magnitudes)
            max_mag = np.max(np.sqrt(U_3d**2 + V_3d**2 + W_3d**2))
            scale = 0.4 / (max_mag + 1e-10)
            U_3d = U_3d * scale
            V_3d = V_3d * scale
            W_3d = W_3d * scale

            fig = self.plot_manager.create_plotly_figure(width=1000, height=800)

            # Surface with color based on vector magnitude
            vec_mag = np.sqrt(U_3d**2 + V_3d**2 + W_3d**2)
            fig.add_trace(
                go.Surface(
                    x=X, y=Y, z=Z,
                    surfacecolor=vec_mag,
                    colorscale='Plasma',
                    opacity=0.85,
                    showscale=True,
                    colorbar=dict(title='Vector Magnitude', len=0.5, y=0.75)
                )
            )

            # Vector field as cones (more arrows for better visualization)
            skip = 3 if manifold_type != 2 else 2
            x_flat = X[::skip, ::skip].flatten()
            y_flat = Y[::skip, ::skip].flatten()
            z_flat = Z[::skip, ::skip].flatten()
            u_flat = U_3d[::skip, ::skip].flatten()
            v_flat = V_3d[::skip, ::skip].flatten()
            w_flat = W_3d[::skip, ::skip].flatten()

            # Color cones by magnitude
            mag_flat = np.sqrt(u_flat**2 + v_flat**2 + w_flat**2)

            fig.add_trace(
                go.Cone(
                    x=x_flat,
                    y=y_flat,
                    z=z_flat,
                    u=u_flat,
                    v=v_flat,
                    w=w_flat,
                    sizemode='absolute',
                    sizeref=0.15,
                    colorscale='Viridis',
                    cmin=0,
                    cmax=np.max(mag_flat),
                    showscale=False,
                    anchor='tail'
                )
            )

            fig.update_layout(
                title=f'Hodge Decomposition on {manifold_name}: {title}',
                scene=dict(
                    xaxis_title='X',
                    yaxis_title='Y',
                    zaxis_title='Z',
                    aspectmode='data',
                    camera=dict(
                        eye=dict(x=1.5, y=1.5, z=1.2)
                    )
                )
            )

            fig.show()

widget = HarmonicFormsWidget(title="Hodge Decomposition: Exact, Co-Exact, and Harmonic Forms")
widget.display()


## 5. Geometric Optics & Rays

When wave equations operate at high frequencies (short wavelengths), the wave nature suppressed, and we recover the geometry of rays.

### 5.1 The Eikonal Equation

Starting with the wave equation $\nabla^2 u - \frac{1}{c^2} \partial_t^2 u = 0$, we insert the WKB ansatz $u = A e^{i k \phi}$. The leading order term yields the nonlinear **Eikonal Equation** for the phase $\phi$:

$$|\nabla \phi|^2 = \frac{1}{c^2}$$

This is a first-order PDE solvable by the Method of Characteristics. The characteristics are the **Light Rays** (geodesics) of the system.

### 5.2 Caustics

While rays usually travel in straight lines or smooth curves, they can focus and cross. The envelope where rays bunch together is a **Caustic**. At these loci, the ray approximation breaks down (amplitude $\to \infty$), and we must return to the full wave theory (or the Airy functions of Level 1).


In [ ]:
# [Ray Tracing & Caustics. An interactive domain with a variable refractive index (e.g., a lens). Users trace rays that bend and focus, forming a visible bright caustic curve (like light in a coffee cup).]

class RayTracingWidget(InteractiveWidget):
    def _setup_widgets(self):
        self.sliders['lens_shape'] = ParameterSlider(
            name='lens_shape',
            value=1.0,
            min_val=0.5,
            max_val=2.0,
            step=0.1,
            description='Lens Curvature'
        )
        self.sliders['num_rays'] = ParameterSlider(
            name='num_rays',
            value=10,
            min_val=5,
            max_val=20,
            step=1,
            description='Number of Rays'
        )

    def _update_plot(self, change=None):
        with self.output:
            self.output.clear_output(wait=True)
            params = self.get_parameters()
            curvature = float(params['lens_shape'])
            num_rays = int(params['num_rays'])

            # Domain: 2D grid for refractive index
            x = np.linspace(-2, 2, 100)
            y = np.linspace(-2, 2, 100)
            X, Y = np.meshgrid(x, y)
            dx = x[1] - x[0]
            dy = y[1] - y[0]

            # Refractive index n(x,y): higher in lens region
            # Lens: circular with n=1.5, outside n=1
            n = 1.0 + 0.5 * np.exp(-curvature * (X**2 + Y**2))

            # Trace rays: from left, parallel incoming
            ray_starts_y = np.linspace(-1.0, 1.0, num_rays)
            ray_paths = []

            for ys in ray_starts_y:
                # Solve ray equation: d/ds (n dr/ds) = grad n
                # Approximate with small steps (Euler method)
                pos = np.array([-2.0, ys], dtype=float)
                # Initial direction (normalized)
                dir_vec = np.array([1.0, 0.0], dtype=float)
                path = [pos.copy()]

                ds = 0.05
                max_steps = 150

                for step in range(max_steps):
                    # Find grid indices
                    ix = int(np.clip((pos[0] - x[0]) / dx, 0, len(x) - 1))
                    iy = int(np.clip((pos[1] - y[0]) / dy, 0, len(y) - 1))

                    # Get n and grad n at current position
                    curr_n = n[iy, ix]

                    # Compute gradient using central differences
                    ix_p = min(ix + 1, len(x) - 1)
                    ix_m = max(ix - 1, 0)
                    iy_p = min(iy + 1, len(y) - 1)
                    iy_m = max(iy - 1, 0)

                    grad_n_x = (n[iy, ix_p] - n[iy, ix_m]) / (2 * dx)
                    grad_n_y = (n[iy_p, ix] - n[iy_m, ix]) / (2 * dy)
                    grad_n = np.array([grad_n_x, grad_n_y])

                    # Ray equation: d/ds (n * dr/ds) = grad n
                    # Update direction
                    # Simplified: dir_new ≈ dir + (ds/n) * (grad_n - (dir·grad_n/|dir|²) * dir)
                    dir_norm_sq = np.dot(dir_vec, dir_vec)
                    if dir_norm_sq > 1e-10:
                        proj = np.dot(grad_n, dir_vec) / dir_norm_sq
                        dir_vec = dir_vec + (ds / curr_n) * (grad_n - proj * dir_vec)
                        # Normalize direction
                        dir_norm = np.linalg.norm(dir_vec)
                        if dir_norm > 1e-10:
                            dir_vec = dir_vec / dir_norm

                    # Update position
                    pos = pos + ds * dir_vec
                    path.append(pos.copy())

                    # Stop if outside domain
                    if pos[0] > 2.0 or abs(pos[1]) > 2.0:
                        break

                ray_paths.append(np.array(path))

            fig = self.plot_manager.create_plotly_figure(width=900, height=700)

            # Refractive index heatmap
            fig.add_trace(
                go.Heatmap(
                    x=x, y=y, z=n.T,
                    colorscale='Viridis',
                    colorbar=dict(title='Refractive Index $n$'),
                    name='Refractive Index'
                )
            )

            # Plot rays
            for i, path in enumerate(ray_paths):
                fig.add_trace(
                    go.Scatter(
                        x=path[:, 0],
                        y=path[:, 1],
                        mode='lines',
                        name=f'Ray {i+1}' if i < 5 else None,
                        line=dict(
                            color='white',
                            width=2
                        ),
                        showlegend=(i < 5),
                        hoverinfo='skip'
                    )
                )

            fig.update_layout(
                title=f'Ray Tracing through Lens (Curvature={curvature:.1f})<br>Rays Focus to Form Caustic',
                xaxis_title='$x$',
                yaxis_title='$y$',
                xaxis_range=[-2, 2],
                yaxis_range=[-2, 2],
                hovermode='closest'
            )

            fig.show()

widget = RayTracingWidget(title="Ray Tracing and Caustic Formation")
widget.display()


---

### References

* **Airy, G. B.** (1838). *On the intensity of light in the neighbourhood of a caustic*.

* **Cartan, É.** (1899). *Sur certaines expressions différentielles et le problème de Pfaff*.

* **Cauchy, A. L.** (1827). *De la pression ou tension dans un corps solide*.

* **Hamilton, W. R.** (1832). *Third supplement to an essay on the theory of systems of rays*.

* **Hodge, W. V. D.** (1941). *The Theory and Applications of Harmonic Integrals*.

* **Kruzhkov, S. N.** (1970). *First order quasilinear equations in several independent variables*.

* **Lax, P. D.** (1957). *Hyperbolic systems of conservation laws II*.

* **Levi-Civita, T.** (1917). *Nozione di parallelismo in una varietà qualunque*.
